# Naive Bayes - Mushroom Classification
Goal is to predict the class of mushrooms, given some features of the mushrooms.
We will use Naive Bayes Model for this classification.


### Load the Dataset




In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split



In [3]:
df = pd.read_csv('../Datasets/Mushrooms/mushrooms.csv')
df.head(n=10)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
5,e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g
6,e,b,s,w,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,n,m
7,e,b,y,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,s,m
8,p,x,y,w,t,p,f,c,n,p,...,s,w,w,p,w,o,p,k,v,g
9,e,b,s,y,t,a,f,c,b,g,...,s,w,w,p,w,o,p,k,s,m


In [39]:
df.shape

(8124, 23)

### Encode the Categorial Data into Numerical Data

In [4]:
le = LabelEncoder()
#Applies transformation on each columns
ds = df.apply(le.fit_transform)


In [5]:
ds.head()
print(type(ds))

<class 'pandas.core.frame.DataFrame'>


In [6]:
data = ds.values
print(data.shape)
print(type(data))
print(data[:5,:])

data_y = data[:,0]
data_x = data[:,1:]

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


### Break the Data into train and test

In [7]:
x_train,x_test,y_train,y_test = train_test_split(data_x,data_y,test_size=0.2)



In [8]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)


(6499, 22) (6499,)
(1625, 22) (1625,)


In [9]:
np.unique(y_train)

array([0, 1], dtype=int64)

# Building Our Classifier !

In [10]:
a = np.array([0,5,5,1,1,1,0,1])


In [11]:
def prior_prob(y_train,label):
    
    total_examples = y_train.shape[0]
    class_examples = np.sum(y_train==label)
    
    return (class_examples)/float(total_examples)
        
    

In [12]:
#y = np.array([0,5,5,1,1,1,1,0,0,0])

#prior_prob(y,5)

In [13]:
def cond_prob(x_train,y_train,feature_col,feature_val,label):
    
    x_filtered = x_train[y_train==label]
    numerator = np.sum(x_filtered[:,feature_col]==feature_val)
    denominator = np.sum(y_train==label)
    
    return numerator/float(denominator)



### Next Step : Compute Posterior Prob for each test example and make predictions

In [14]:
np.unique(y_train)

array([0, 1], dtype=int64)

In [15]:
def predict(x_train,y_train,xtest):
    """Xtest is a single testing point, n features"""
    
    classes = np.unique(y_train)
    n_features = x_train.shape[1]
    post_probs = [] # List of prob for all classes and given a single testing point
    #Compute Posterior for each class
    for label in classes:
        
        #Post_c = likelihood*prior
        likelihood = 1.0
        for f in range(n_features):
            cond = cond_prob(x_train,y_train,f,xtest[f],label)
            likelihood *= cond 
            
        prior = prior_prob(y_train,label)
        post = likelihood*prior
        post_probs.append(post)
        
    pred = np.argmax(post_probs)
    return pred

In [15]:
output = predict(x_train,y_train,x_test[1])
print(output)
print(y_test[1])
    

1
1


In [22]:
def score(x_train,y_train,x_test,y_test):

    pred = []
    for i in range(x_test.shape[0]):
        pred_label = predict(x_train,y_train,x_test[i])
        pred.append(pred_label) # <===Correction
    
    pred = np.array(pred)
    
    accuracy = np.sum(pred==y_test)/y_test.shape[0]
    return accuracy

In [23]:
print(score(x_train,y_train,x_test,y_test))

0.9975384615384615


### ..and its works well!

